In [ ]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    save_raw_data,
    plot_summary,
)
import os
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from tifffile import imwrite
from tqdm import tqdm

%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 300

# Welcome to the video analysis pipeline module!

TODO: Fix dropbox uploads in flux_extract!!!
TODO: Streamline actions taken in flux_extract!!!
TODO: Fix videoInfo being downloaded to analysis!!!

Use this notebook to extract kymographs and transport flows from raw video TIFFs. The first step is to supply some file parameters to make sure that the measurements are connected to real metrics and that the segmentation method is correct.

Important to note: 
Seg-Tresh is the limit of what percentage of the segmented image should be hypha. 

In [ ]:
### This is where the video folder in question is supplied. If there is an excel file one level higher,
### it will pull the relevant parameters from that.

img_address = "/gpfs/scratch1/shared/amftrackflow/videos/MYRISTATE/DATA/2_weeks/20230313_Plate031/02/"

In [ ]:
### Logging     means that you will see print messages documenting the progress.
### vid_type    is either 'BRIGHT' or 'FLUO' for brightfield and fluorescence microscopy respectively.
###                 Will be overruled if excel file is present
### fps         is the frames per second of the video
###                 Will be overruled if excel file is present
### binning     is whether the video was imaged at full resolution, or half resolution
###                 Will be overruled if excel file is present
### filter_step is not a known value
### seg_thresh  is the percentage coverage that the segmentation must achieve
### show_seg    if true will plot the segmentation image


test_video = Kymo_video_analysis(
    img_address,
    logging=True,
    vid_type=None,
    fps=None,
    binning=None,
    filter_step=60,
    seg_thresh=20,
    show_seg=True,
)

If the segmentation is satisfactory, you can proceed with plotting the edges that will be analyzed. 

In [ ]:
edge_list = test_video.edge_objects

### The print statement will give you the edge indices such that you can select which edges to analyze.
# print('\n To work with individual edges, here is a list of their indices:')
# for i, edge in enumerate(edge_list):
#     print('edge {}, {}'.format(i, edge.edge_name))

### Target length here determines the *width* of the analysis box.
### Too shallow, and you won't capture the entire hypha, too big and the analysis is plagued with background.
target_length = int(1.9 * test_video.magnification)

test_video.plot_extraction_img(target_length=target_length, save_img=True)
edges_total = test_video.edge_objects

## Below function will make an mp4 from the tiff files, can take a while, if you prefer to skip, that's fine.

In [ ]:
test_video.makeVideo()

## This is where you select the edges that you want to analyze.
Bin number creates a range of how the edges will be partitioned along their length.

In [ ]:
edge_seq = [5]
# bin_nrs = np.arange(1,5)

In [ ]:
bin_nr = 1
edge_objs = [edges_total[i] for i in edge_seq]
img_seq = np.arange(len(edge_objs[0].video_analysis.selection_file))
kymos = []

In [ ]:
### Exponential fit is performed to adjust for photobleaching
# exp_fit    = test_video.fit_backgr(img_seq, plots=True)

### A video and single frame will be made of each edge to get a closer look
# edge_pic   = edge.view_edge(img_frame=40 ,save_im=True, target_length = target_length)
# edge_video = edge.view_edge(img_frame = img_seq, save_im=True, quality = 6, target_length=target_length)

### Analysis consists of extracting a kymograph, doing fourier filtering, then extracting speeds, then transport.
### plots in each function turn on plt plots.
# for bin_nr in bin_nrs:
for edge in edge_objs:
    space_res = edge.video_analysis.space_pixel_size
    time_res = edge.video_analysis.time_pixel_size
    video_kymos = edge.extract_multi_kymo(
        bin_nr, target_length=target_length, kymo_adj=False
    )
    kymos.append(video_kymos)
    imshow_extent = [
        0,
        edge.video_analysis.space_pixel_size * edge.kymos[0].shape[1],
        edge.video_analysis.time_pixel_size * edge.kymos[0].shape[0],
        0,
    ]
    kymos_lefts, kymos_rights = edge.fourier_kymo(bin_nr, save_im=True, save_array=True)
    speeds, times = edge.test_GST(
        15,
        w_start=3,
        C_thresh=0.95,
        C_thresh_falloff=0.001,
        blur_size=5,
        preblur=True,
        speed_thresh=50,
        plots=False,
    )
    net_trans = edge.extract_transport(
        noise_thresh=0.15, plots=False, save_im=True, save_flux_array=True, margin=5
    )

#     fig, ax = plt.subplots(2, figsize=(8,8*1.7))
#     ax[0].imshow(video_kymos[0], extent=imshow_extent, aspect='auto')
#     ax[1].imshow(net_trans, cmap = 'coolwarm', aspect='auto', extent = imshow_extent)
#     fig.tight_layout()

In [ ]:
plot_summary(edge_objs)

In [ ]:
for edge in edge_objs:
    widths = edge.get_widths(img_frame=40, save_im=True, target_length=200, step=80)
    print(np.mean(widths))

    fig, ax = plt.subplots()
    ax.plot(widths)
    ax.set_ylim([0, 20])

# 9.427204
# 9.574729
# 6.1030574

## Experiment zone!!!


In [ ]:
edge_table = {
    "edge_name": [],
    "edge_length": [],
    "straight_length": [],
    "speed_max": [],
    "speed_min": [],
    "flux_avg": [],
    "flux_min": [],
    "flux_max": [],
}
data_edge = pd.DataFrame(data=edge_table)

for edge in edge_objs:
    straight_len = (
        np.linalg.norm(
            (edge.segments[0][0] + edge.segments[0][1]) / 2
            - (edge.segments[-1][0] + edge.segments[-1][1]) / 2
        )
        * space_res
    )

    new_row = pd.DataFrame(
        [
            {
                "edge_name": f"{edge.edge_name}",
                "edge_length": space_res * edge.kymos[0].shape[1],
                "straight_length": straight_len,
                "speed_max": np.nanpercentile(speeds[0][1], 97),
                "speed_min": np.nanpercentile(speeds[0][0], 3),
                "flux_avg": np.nanmean(net_trans),
                "flux_min": np.nanpercentile(net_trans, 3),
                "flux_max": np.nanpercentile(net_trans, 97),
            }
        ]
    )

    data_edge = pd.concat([data_edge, new_row])

data_edge

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(9, 3.3))
ax[0].imshow(video_kymos[0], cmap="gray", vmin=0, aspect="auto", extent=imshow_extent)
# ax[0].imshow((kymos_lefts[0]+ kymos_rights[0])/2, cmap='gray', vmin=0, aspect='auto', extent=imshow_extent)
ax[2].imshow(kymos_lefts[0], cmap="gray", vmin=0, aspect="auto", extent=imshow_extent)
ax[1].imshow(kymos_rights[0], cmap="gray", vmin=0, aspect="auto", extent=imshow_extent)

ax[0].set_title("Kymograph")
ax[0].set_xlabel("space $(\mu m)$")
ax[0].set_ylabel("time (s)")
# ax[0][1].set_title("Kymograph (no static)")
# ax[0][1].set_xlabel("space $(\mu m)$")
# ax[0][1].set_ylabel("time (s)")
ax[2].set_title("Right filtered")
ax[2].set_xlabel("space $(\mu m)$")
# ax[1].set_ylabel("time (s)")
ax[1].set_title("Left filtered")
ax[1].set_xlabel("space $(\mu m)$")
# ax[1].set_ylabel("time (s)")
fig.tight_layout()
fig.savefig(
    "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230126_Plate527/20230126_Plate527_03/Analysis/edge (840, 306)/kymographs_graph.png"
)

In [ ]:
kymo_tiff = np.array(
    [video_kymos[0], kymos_lefts[0] + kymos_rights[0], kymos_lefts[0], kymos_rights[0]],
    dtype=np.int16,
)
imwrite(
    "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230126_Plate527/20230126_Plate527_03/Analysis/edge (840, 306)/kymographs_array.tiff",
    kymo_tiff,
    photometric="minisblack",
)

In [ ]:
speedmax = np.nanmax(abs(np.array([speeds[0][0], speeds[0][1]])))
# flux_non_nan = ~(np.isnan(speeds[0][0]) * np.isnan(speeds[0][1]))

fig, ax = plt.subplots(1, 3, figsize=(9, 3.3), layout="constrained")
ax[1].imshow(
    speeds[0][0],
    cmap="coolwarm",
    aspect="auto",
    vmax=speedmax,
    vmin=-speedmax,
    extent=imshow_extent,
)
cbar = ax[2].imshow(
    speeds[0][1],
    cmap="coolwarm",
    aspect="auto",
    vmax=speedmax,
    vmin=-speedmax,
    extent=imshow_extent,
)
cbar_a = fig.colorbar(cbar, ax=ax[0], location="left")
cbar2 = ax[0].imshow(net_trans, cmap="coolwarm", aspect="auto", extent=imshow_extent)
# fig.colorbar(cbar2, ax=ax[1][1])
# ax[1][0].plot(times[0], 1 - np.count_nonzero(np.isnan(speeds[0][0]), axis=1) / len(net_trans[0]), label="Left")
# ax[1][0].plot(times[0], 1 - np.count_nonzero(np.isnan(speeds[0][1]), axis=1) / len(net_trans[0]), label="Right")
# ax[1][0].plot(times[0], 1 - np.count_nonzero(np.isnan(net_trans), axis=1) / len(net_trans[0]), label="Total")

# ax[1][0].set_ylim([0, 1])
# ax[1][0].legend()
# ax[1][0].grid(True)
# ax[1][0].set_title("Speed coverage")
# ax[1][0].set_xlabel("time (s)")
# ax[1][0].set_ylabel("Hypha length coverage (%)")

ax[0].set_title("Left + Right")
ax[0].set_xlabel("space $(\mu m)$")
ax[0].set_ylabel("time (s)")
ax[1].set_title("Speeds left")
ax[1].set_xlabel("space $(\mu m)$")
# ax[0][1].set_ylabel("time (s)")
ax[2].set_title("Speeds right")
ax[2].set_xlabel("space $(\mu m)$")

cbar_a.set_label("speed $(\mu m / s)$")

# ax[1][1].set_ylabel("time (s)")

# fig.tight_layout()

In [ ]:
spds_tiff = np.array([speeds[0][0], speeds[0][1], net_trans], dtype=float)
imwrite(
    "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230126_Plate527/20230126_Plate527_03/Analysis/edge (840, 306)/speeds_flux_array.tiff",
    spds_tiff,
    photometric="minisblack",
)

In [ ]:
total_len = video_kymos[0].shape[1]
split_index = 300
split_invers = total_len - split_index

split_kymo = [video_kymos[0].T[:split_index].T, video_kymos[0].T[split_index:].T]

kymo_edge_list = [Kymo_edge_analysis(kymo=kymo) for kymo in split_kymo]

fig, ax = plt.subplots(3, 3, figsize=(12, 15), sharey="row")
net_speeds = []
net_net_trans = []

for i, kymo_anal in enumerate(tqdm(kymo_edge_list)):
    title = f"({i}, {i+1})"

    imshow_extent = [
        0,
        space_res * kymo_anal.kymo.shape[1],
        time_res * kymo_anal.kymo.shape[0],
        0,
    ]
    kymo_anal.space_pixel_size = space_res
    kymo_anal.time_pixel_size = time_res

    forw_thresh, back_thresh = kymo_anal.fourier_kymo(1, test_plots=False)

    speeds_split, times = kymo_anal.test_GST(
        15,
        w_start=3,
        C_thresh=0.95,
        C_thresh_falloff=0.001,
        blur_size=3,
        preblur=True,
        speed_thresh=50,
        plots=False,
    )
    net_trans_split = kymo_anal.extract_transport(
        noise_thresh=0.15,
        plots=False,
        save_im=False,
        save_flux_array=False,
        save_filters=False,
        margin=5,
    )

    net_net_trans.append(net_trans_split)
    net_speeds.append(speeds_split)
    ax[0][i].imshow(kymo_anal.kymo, extent=imshow_extent, aspect="auto")
    ax[0][i].set_title(f"{['Left','Right'][i]} partition kymo")
    ax[0][i].set_xlabel("space ($\mu m$)")
    ax[0][i].set_ylabel("time (s)")

    ax[1][i].errorbar(
        times[0],
        np.nanmean(speeds_split[0][0], axis=1),
        np.nanstd(speeds_split[0][0], axis=1),
        label="Backward speed",
        errorevery=15,
    )
    ax[1][i].errorbar(
        times[0],
        np.nanmean(speeds_split[0][1], axis=1),
        np.nanstd(speeds_split[0][1], axis=1),
        label="Forward speed",
        errorevery=15,
    )
    ax[1][i].set_title(f"{['Left','Right'][i]} partition mean speed")
    ax[1][i].set_xlabel("time (s)")
    ax[1][i].set_ylabel("speed ($\mu m/s$)")
    ax[1][i].legend()
    ax[1][i].grid(True)

    ax[2][i].plot(times[0], np.nanmean(net_trans_split, axis=1))
    ax[2][i].set_title(f"{['Left','Right'][i]} partition mean flux")
    ax[2][i].set_xlabel("time (s)")
    ax[2][i].set_ylabel("flux ($q \mu m / s$)")
    ax[2][i].grid(True)


# net_speeds = np.array(speeds)
speed_mean = [np.nanmean(speed[0][0], axis=1) for speed in net_speeds]

imshow_extent = [
    0,
    space_res * video_kymos[0].shape[1],
    time_res * video_kymos[0].shape[0],
    0,
]

ax[0][2].imshow(video_kymos[0], extent=imshow_extent, aspect="auto")
ax[0][2].set_title("Full kymo")
ax[1][2].plot(
    times[0], np.nanmean(speeds[0][0], axis=1), c="tab:blue", label="Full kymo backward"
)
ax[1][2].plot(
    times[0],
    np.sum(
        [
            np.nanmean(speed[0][0], axis=1)
            * ([split_index / total_len, split_invers / total_len][i])
            for i, speed in enumerate(net_speeds)
        ],
        axis=0,
    ),
    alpha=0.5,
    c="tab:blue",
    label="Sum of partition backward",
)

ax[1][2].plot(
    times[0],
    np.nanmean(speeds[0][1], axis=1),
    c="tab:orange",
    label="Full kymo forward",
)
ax[1][2].plot(
    times[0],
    np.sum(
        [
            np.nanmean(speed[0][1], axis=1)
            * ([split_index / total_len, split_invers / total_len][i])
            for i, speed in enumerate(net_speeds)
        ],
        axis=0,
    ),
    alpha=0.5,
    c="tab:orange",
    label="Sum of partition forward",
)


ax[1][2].set_title(f"Full mean speed")
ax[1][2].set_xlabel("time (s)")
ax[1][2].set_ylabel("speed ($\mu m/s$)")
ax[1][2].legend()
ax[1][2].grid(True)
ax[2][2].plot(times[0], np.nanmean(net_trans, axis=1), label="Full kymo")
ax[2][2].plot(
    times[0],
    np.sum(
        [
            np.nanmean(net_net_trans_t, axis=1)
            * ([split_index / total_len, split_invers / total_len][i])
            for i, net_net_trans_t in enumerate(net_net_trans)
        ],
        axis=0,
    ),
    alpha=0.5,
    label="Sum of partition",
)
ax[2][2].set_title(f"Full mean flux")
ax[2][2].set_xlabel("time (s)")
ax[2][2].set_ylabel("flux ($q \mu m / s$)")
ax[2][2].legend()
ax[2][2].grid(True)

fig.tight_layout()

In [ ]:
total_len = video_kymos[0].shape[1]
# split_index = 300
# split_invers = total_len - split_index

# split_kymo = [video_kymos[0].T[:split_index].T, video_kymos[0].T[split_index:].T]

# kymo_edge_list = [Kymo_edge_analysis(kymo=kymo) for kymo in split_kymo]

fig, ax = plt.subplots(3, figsize=(7, 7 * 3))
net_speeds = []
net_net_trans = []

# for i, kymo_anal in enumerate(tqdm(kymo_edge_list)):
#     title = f"({i}, {i+1})"

#     imshow_extent = [0, space_res * kymo_anal.kymo.shape[1],
#                 time_res * kymo_anal.kymo.shape[0], 0]
#     kymo_anal.space_pixel_size = space_res
#     kymo_anal.time_pixel_size = time_res

#     forw_thresh, back_thresh = kymo_anal.fourier_kymo(1, test_plots=False)

#     speeds_split, times = kymo_anal.test_GST(15, w_start=3, C_thresh=0.95, C_thresh_falloff = 0.001, blur_size = 3, preblur=True, speed_thresh=50, plots=False)
#     net_trans_split = kymo_anal.extract_transport(noise_thresh=0.15, plots=False, save_im=False, save_flux_array=False, save_filters=False, margin=5)

#     net_net_trans.append(net_trans_split)
#     net_speeds.append(speeds_split)


# net_speeds = np.array(speeds)
speed_mean = [np.nanmean(speed[0][0], axis=1) for speed in net_speeds]

imshow_extent = [
    0,
    space_res * video_kymos[0].shape[1],
    time_res * video_kymos[0].shape[0],
    0,
]

ax[0].imshow(video_kymos[0], extent=imshow_extent, aspect="auto")
ax[0].set_title("Full kymo")
ax[1].errorbar(
    times[0],
    np.nanmean(speeds[0][0], axis=1),
    np.nanstd(speeds[0][0], axis=1),
    c="tab:blue",
    label="Full kymo backward",
    errorevery=10,
)
# ax[1].plot(times[0], np.sum([np.nanmean(speed[0][0], axis=1)*([split_index/total_len, split_invers/total_len][i]) for i, speed in enumerate(net_speeds)], axis=0), alpha=0.5, c='tab:blue', label='Sum of partition backward')

ax[1].errorbar(
    times[0],
    np.nanmean(speeds[0][1], axis=1),
    np.nanstd(speeds[0][1], axis=1),
    c="tab:orange",
    label="Full kymo forward",
    errorevery=10,
)
# ax[1].plot(times[0], np.sum([np.nanmean(speed[0][1], axis=1)*([split_index/total_len, split_invers/total_len][i]) for i, speed in enumerate(net_speeds)], axis=0), alpha=0.5, c='tab:orange', label='Sum of partition forward')


ax[1].set_title(f"Full mean speed")
ax[1].set_xlabel("time (s)")
ax[1].set_ylabel("speed ($\mu m/s$)")
ax[1].legend()
ax[1].grid(True)
ax[2].errorbar(
    times[0],
    np.nanmean(net_trans, axis=1),
    np.nanstd(net_trans, axis=1),
    label="Full kymo",
    errorevery=10,
)
# ax[2].plot(times[0], np.sum([np.nanmean(net_net_trans_t, axis=1)*([split_index/total_len, split_invers/total_len][i]) for i, net_net_trans_t in enumerate(net_net_trans)], axis=0), alpha=0.5, label='Sum of partition')
ax[2].set_title(f"Full mean flux")
ax[2].set_xlabel("time (s)")
ax[2].set_ylabel("flux ($q \mu m / s$)")
ax[2].legend()
ax[2].grid(True)

fig.tight_layout()

In [ ]:
data_table = {
    "times": times[0],
    "speed_right_mean": np.nanmean(speeds[0][1], axis=1),
    "speed_left_mean": np.nanmean(speeds[0][0], axis=1),
    "speed_right_std": np.nanstd(speeds[0][0], axis=1),
    "speed_left_std": np.nanstd(speeds[0][1], axis=1),
    "flux_mean": np.nanmean(net_trans, axis=1),
    "flux_std": np.nanstd(net_trans, axis=1),
    "flux_coverage": np.count_nonzero(net_trans, axis=1) / len(net_trans[0]),
    "speed_left_coverage": 1
    - np.count_nonzero(np.isnan(speeds[0][0]), axis=1) / len(net_trans[0]),
    "speed_right_coverage": 1
    - np.count_nonzero(np.isnan(speeds[0][1]), axis=1) / len(net_trans[0]),
}

data_out = pd.DataFrame(data=data_table)
data_out

In [ ]:
# fig, ax = plt.subplots(len(edge_objs), bin_nr, figsize=(6, 12), sharey='row')
# count = 0

# bin_space = np.linspace(0,1, bin_nr+1)

# for i in range(len(edge_objs)):
#     for j in range(bin_nr):
#         if len(edge_objs) > 1:
#             if bin_nr > 1:
#                 ax[i][j].imshow(kymo[i][j])
#                 ax[i][j].set_title(f"Kymo [{bin_space[j]}-{bin_space[j+1]}] of edge {edge_objs[i].edge_name}")
#                 ax[i][j].set_xlabel("space ($x$)")
#                 ax[i][j].set_ylabel("time ($t$)")
#             elif bin_nr == 1:
#                 ax[i].imshow(kymo[i][0])
#                 ax[i].set_title(f"Kymo of edge {edge_objs[i].edge_name}")
#                 ax[i].set_xlabel("space ($x$)")
#                 ax[i].set_ylabel("time ($t$)")

#         else:
#             if bin_nr == 1:
#                 ax = [ax]
#             ax[j].imshow(kymo[i][j])
#             ax[j].set_title(f"Kymo [{bin_space[j]}-{bin_space[j+1]}]")
#             ax[j].set_xlabel("space ($x$)")
#             ax[j].set_ylabel("time ($t$)")

# fig.tight_layout()
# # plt.show()

In [ ]:
# fig, ax = plt.subplots(1, bins)
# for i in range(1, 30):
#     for j in range(bins):
#         corr = signal.correlate(kymo[1][0], kymo[1][bins_space[j]:bins_space[j+1]][i])
#         corr /= np.max(corr)
#         lags = signal.correlation_lags(len(kymo[0][0]), len(kymo[0][bins_space[j]:bins_space[j+1]][i]))
#         ax[j].plot(lags, corr)
# plt.show()

In [ ]:
# fig, ax = plt.subplots(len(edge_objs), bin_nr, figsize=(6, 12), sharey='row')
# count = 0

# bin_space = np.linspace(0,1, bin_nr+1)

# for i in range(len(edge_objs)):
#     for j in range(bin_nr):
#         if len(edge_objs) > 1:
#             if bin_nr > 1:
#                 ax[i][j].imshow(kymo[i][j])
#                 ax[i][j].set_title(f"Kymo [{bin_space[j]}-{bin_space[j+1]}] of edge {edge_objs[i].edge_name}")
#                 ax[i][j].set_xlabel("space ($x$)")
#                 ax[i][j].set_ylabel("time ($t$)")
#             elif bin_nr == 1:
#                 ax[i].imshow(kymo[i][0])
#                 ax[i].set_title(f"Kymo of edge {edge_objs[i].edge_name}")
#                 ax[i].set_xlabel("space ($x$)")
#                 ax[i].set_ylabel("time ($t$)")

#         else:
#             if bin_nr == 1:
#                 ax = [ax]
#             ax[j].imshow(kymo[i][j])
#             ax[j].set_title(f"Kymo [{bin_space[j]}-{bin_space[j+1]}]")
#             ax[j].set_xlabel("space ($x$)")
#             ax[j].set_ylabel("time ($t$)")

# fig.tight_layout()
# # plt.show()